# Keras 训练中使用TensorBoard

Keras在回调函数中内置Tensorboard函数：

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
tf.keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=0,
    embeddings_metadata=None
)

* **Scalars** 显示了如何将loss与每个时间段改变。还可以使用它来跟踪训练速度，学习率和其他标量值。
* **Graphs** 进行了可视化模型。在这种情况下，将现实层的Keras图，确保模型构建
* **Distributions**和**Histograms**：显示张量随时间的分布。这对于可视化权重和偏差并验证它们是否以预期的方式变化很有用

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import datetime
print(tf.__version__)
print(np.__version__)

2.2.0
1.18.5


In [5]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
)

IndexError: list index out of range

In [6]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False